In [ ]:
import numpy as np
import pandas as pd
import qubx  # type: ignore
from qubx.core.basics import DataType

%qubxd

%load_ext autoreload
%autoreload 2

# - - - - - - - - - -
from qubx.data.transformers import PandasFrame, TypedRecords, OHLCVSeries, TickSeries
from qubx.data.storages.csv import CsvStorage


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.84
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


## Basic CSV storage

First create instance of storage

In [4]:
s = CsvStorage("~/devs/Qubx/tests/data/storages/csv")

Every storage provides information about exchanges 

In [6]:
s.get_exchanges()

['BINANCE.UM']

And types of instruments (SWAP, FUTURES, SPOT etc)

In [110]:
s.get_market_types("BINANCE.UM")

['SWAP']

## Reader

For reading data it needs to get dedicated reader for exchange and market type

### Reading single instrument

In [8]:
bnc_swap = s.get_reader("BINANCE.UM", "SWAP")

Reader can provide information about what type of data this storage has for symbol

In [10]:
bnc_swap.get_data_types("BTCUSDT"), bnc_swap.get_data_types("ETHUSDT")

(['ohlc(1h)', quote], ['ohlc(1h)', trade])

Also it's possible to get list of symbols which have given data in this storage

In [11]:
bnc_swap.get_data_id("ohlc(1h)")

['ETHUSDT', 'BCHUSDT', 'BTCUSDT', 'LTCUSDT', 'AAVEUSDT']

In [12]:
bnc_swap.get_data_id("quote")

['BTCUSDT']

Can provide time ranges for symbol and data type

In [17]:
print(bnc_swap.get_time_range("BTCUSDT", DataType.OHLC["1h"]))
print(bnc_swap.get_time_range("BTCUSDT", DataType.QUOTE))

(numpy.datetime64('2023-06-01T00:00:00'), numpy.datetime64('2023-08-01T00:00:00'))
(numpy.datetime64('2017-08-24T13:01:12'), numpy.datetime64('2017-08-24T13:01:59'))


Reading single symbol data

In [25]:
ds = bnc_swap.read("BTCUSDT", DataType.OHLC['1h'], None, None)
ds

BTCUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-08-01 00:00:00]

ds - just simple container of raw data (not preprocessed) from the storage.

In [26]:
ds.raw[:10]

array([[Timestamp('2023-06-01 00:00:00'), 27201.1, 27347.9, 27008.1, 27059.8, 33639.568, 914441790.0, 256082, 15501.039, 421503008.0],
       [Timestamp('2023-06-01 01:00:00'), 27059.8, 27155.7, 27045.7, 27065.5, 17636.072, 478001888.0, 141280, 8794.62, 238385216.0],
       [Timestamp('2023-06-01 02:00:00'), 27065.5, 27106.5, 26621.3, 26705.1, 42682.676, 1144205950.0, 268982, 17221.14, 461549856.0],
       [Timestamp('2023-06-01 03:00:00'), 26705.0, 26829.0, 26600.0, 26768.6, 27103.262, 724444350.0, 219415, 14888.465, 398015552.0],
       [Timestamp('2023-06-01 04:00:00'), 26768.6, 26813.6, 26742.0, 26797.2, 9880.983, 264626432.0, 94780, 5605.699, 150137440.0],
       [Timestamp('2023-06-01 05:00:00'), 26797.1, 26858.7, 26771.8, 26842.9, 8112.849, 217591744.0, 91324, 4429.024, 118795592.0],
       [Timestamp('2023-06-01 06:00:00'), 26843.0, 26876.6, 26762.0, 26777.6, 11210.559, 300753344.0, 105171, 5177.757, 138924640.0],
       [Timestamp('2023-06-01 07:00:00'), 26777.6, 26856.0, 2675

In [117]:
# so we can easily do any numpy operations on it if need
np.mean(np.array(ds.raw[:10])[:, 1:])

80728452.46386667

Read quotes data (as raw arrays)

In [27]:
bnc_swap.read("BTCUSDT", DataType.QUOTE, None, None).raw[:10]

array([[Timestamp('2017-08-24 13:01:12'), 10.1, 10.0, 200, 100],
       [Timestamp('2017-08-24 13:01:13'), 10.1, 10.01, 20000, 100],
       [Timestamp('2017-08-24 13:01:14'), 10.13, 10.03, 200, 100],
       [Timestamp('2017-08-24 13:01:15'), 10.12, 10.01, 20000, 100],
       [Timestamp('2017-08-24 13:01:16'), 10.11, 10.1, 200, 100],
       [Timestamp('2017-08-24 13:01:17'), 10.19, 10.16, 200, 100],
       [Timestamp('2017-08-24 13:01:18'), 10.19, 10.16, 200, 10000],
       [Timestamp('2017-08-24 13:01:19'), 10.19, 10.15, 200, 100],
       [Timestamp('2017-08-24 13:01:20'), 10.18, 10.15, 20000, 10000],
       [Timestamp('2017-08-24 13:01:21'), 10.18, 10.15, 200, 10000]], dtype=object)

Read trades data (as raw arrays)

In [30]:
bnc_swap.read("ETHUSDT", DataType.TRADE, None, None).raw[:10]

array([[3920436870, 3054.91, 0.1, 305.491, 1713571200068, True],
       [3920436871, 3054.91, 0.144, 439.90704, 1713571200199, True],
       [3920436872, 3054.91, 0.328, 1002.01048, 1713571200199, True],
       [3920436873, 3054.91, 0.328, 1002.01048, 1713571200202, True],
       [3920436874, 3054.91, 0.037, 113.03167, 1713571200205, True],
       [3920436875, 3054.92, 0.007, 21.38444, 1713571200209, False],
       [3920436876, 3054.91, 0.328, 1002.01048, 1713571200225, True],
       [3920436877, 3054.91, 0.917, 2801.35247, 1713571200279, True],
       [3920436878, 3054.92, 0.027, 82.48284, 1713571200297, False],
       [3920436879, 3054.92, 0.174, 531.55608, 1713571200297, False]], dtype=object)

Reading data by chunks is also possible

In [31]:
for w in bnc_swap.read("BTCUSDT", DataType.OHLC['1h'], None, None, chunksize=100):
    print(w)

BTCUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-05 03:00:00]
BTCUSDT(ohlc(1h))[2023-06-05 04:00:00 : 2023-06-09 07:00:00]
BTCUSDT(ohlc(1h))[2023-06-09 08:00:00 : 2023-06-13 11:00:00]
BTCUSDT(ohlc(1h))[2023-06-13 12:00:00 : 2023-06-17 15:00:00]
BTCUSDT(ohlc(1h))[2023-06-17 16:00:00 : 2023-06-21 19:00:00]
BTCUSDT(ohlc(1h))[2023-06-21 20:00:00 : 2023-06-25 23:00:00]
BTCUSDT(ohlc(1h))[2023-06-26 00:00:00 : 2023-06-30 03:00:00]
BTCUSDT(ohlc(1h))[2023-06-30 04:00:00 : 2023-07-04 07:00:00]
BTCUSDT(ohlc(1h))[2023-07-04 08:00:00 : 2023-07-08 11:00:00]
BTCUSDT(ohlc(1h))[2023-07-08 12:00:00 : 2023-07-12 15:00:00]
BTCUSDT(ohlc(1h))[2023-07-12 16:00:00 : 2023-07-16 19:00:00]
BTCUSDT(ohlc(1h))[2023-07-16 20:00:00 : 2023-07-20 23:00:00]
BTCUSDT(ohlc(1h))[2023-07-21 00:00:00 : 2023-07-25 03:00:00]
BTCUSDT(ohlc(1h))[2023-07-25 04:00:00 : 2023-07-29 07:00:00]
BTCUSDT(ohlc(1h))[2023-07-29 08:00:00 : 2023-08-01 00:00:00]


In [118]:
try:
    print(bnc_swap.read("ETHUSDT", DataType.OHLC["1Min"]).raw[:10])
except Exception as e:
    print(e)

Can't find any csv data for 'ETHUSDT' of 'ohlc(1Min)' in /home/quant0/devs/Qubx/tests/data/storages/csv/BINANCE.UM/SWAP !


### Reading multiple symbols

New approach supports reading multiple symbols per one reading operation

In [119]:
bnc_swap = s.get_reader("BINANCE.UM", "SWAP")

In [120]:
symbols = bnc_swap.get_data_id("ohlc(1h)")
symbols

['ETHUSDT', 'BCHUSDT', 'BTCUSDT', 'LTCUSDT', 'AAVEUSDT']

In [121]:
mdr = bnc_swap.read(symbols, DataType.OHLC["1h"], "2023-06-01 00:00", "+12h")
mdr

-[MULTI DATA]-
	 - ETHUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 12:00:00]
	 - BCHUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 12:00:00]
	 - BTCUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 12:00:00]
	 - LTCUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 12:00:00]
	 - AAVEUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 12:00:00]

In [122]:
mdr["BTCUSDT"]

BTCUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 12:00:00]

mdr - just container for raw data results

### Reading multi by chanks

It's possible to read multiple symbols by chunks

In [123]:
raws_iters = bnc_swap.read(["BTCUSDT", "ETHUSDT", "LTCUSDT"], DataType.OHLC["1h"], "2023-06-01 00:00", "+12h", chunksize=10)

for rw in raws_iters:
    print(rw)

-[MULTI DATA]-
	 - BTCUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 09:00:00]
	 - ETHUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 09:00:00]
	 - LTCUSDT(ohlc(1h))[2023-06-01 00:00:00 : 2023-06-01 09:00:00]
-[MULTI DATA]-
	 - BTCUSDT(ohlc(1h))[2023-06-01 10:00:00 : 2023-06-01 12:00:00]
	 - ETHUSDT(ohlc(1h))[2023-06-01 10:00:00 : 2023-06-01 12:00:00]
	 - LTCUSDT(ohlc(1h))[2023-06-01 10:00:00 : 2023-06-01 12:00:00]


## Transformation

After data is fetched it's possible to apply different types of transformations on it.
Now 4 transformations are supported:
 - **PandasFrame()**: convert raw data to pandas DataFrame
 - **TypedRecords()**: recognize raw data and convert it into Qubx timestamped objects (Quote, Trade, Bar, ....)
 - **OHLCSeries()**: convert ohlc data into Qubx OHLCV series 
 - **TickSeries()**: convert ohlc data into series of simulated Quotes/Trades

In [47]:
s = CsvStorage("~/devs/Qubx/tests/data/storages/csv")
bnc_swap = s["BINANCE.UM", "SWAP"]

raw1 = bnc_swap.read("BTCUSDT", DataType.QUOTE, None, None)

In [51]:
raw1.transform(PandasFrame()).head()

,ask,bid,askvol,bidvol
time,,,,
2017-08-24 13:01:12,10.1,10.0,200,100
2017-08-24 13:01:13,10.1,10.01,20000,100
2017-08-24 13:01:14,10.13,10.03,200,100
2017-08-24 13:01:15,10.12,10.01,20000,100
2017-08-24 13:01:16,10.11,10.1,200,100


In [52]:
raw1.transform(PandasFrame(True)).head()

,,ask,bid,askvol,bidvol
time,symbol,,,,
2017-08-24 13:01:12,BTCUSDT,10.1,10.0,200,100
2017-08-24 13:01:13,BTCUSDT,10.1,10.01,20000,100
2017-08-24 13:01:14,BTCUSDT,10.13,10.03,200,100
2017-08-24 13:01:15,BTCUSDT,10.12,10.01,20000,100
2017-08-24 13:01:16,BTCUSDT,10.11,10.1,200,100


Read data and convert to list of Qubx types objects

In [ ]:
raw1.transform(TypedRecords())[:10]

[[2017-08-24T13:01:12.000000000]	10.00000 (100.00) | 10.10000 (200.00),
 [2017-08-24T13:01:13.000000000]	10.01000 (100.00) | 10.10000 (20000.00),
 [2017-08-24T13:01:14.000000000]	10.03000 (100.00) | 10.13000 (200.00),
 [2017-08-24T13:01:15.000000000]	10.01000 (100.00) | 10.12000 (20000.00),
 [2017-08-24T13:01:16.000000000]	10.10000 (100.00) | 10.11000 (200.00),
 [2017-08-24T13:01:17.000000000]	10.16000 (100.00) | 10.19000 (200.00),
 [2017-08-24T13:01:18.000000000]	10.16000 (10000.00) | 10.19000 (200.00),
 [2017-08-24T13:01:19.000000000]	10.15000 (100.00) | 10.19000 (200.00),
 [2017-08-24T13:01:20.000000000]	10.15000 (10000.00) | 10.18000 (20000.00),
 [2017-08-24T13:01:21.000000000]	10.15000 (10000.00) | 10.18000 (200.00)]

Read OHLC and convert to pandas

In [59]:
raw2 = bnc_swap.read("BTCUSDT", DataType.OHLC["1h"], None, None)
raw2.transform(PandasFrame())

,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume
timestamp,,,,,,,,,
2023-06-01 00:00:00,27201.1,27347.9,27008.1,27059.8,33639.568,914441790.0,256082,15501.039,421503008.0
2023-06-01 01:00:00,27059.8,27155.7,27045.7,27065.5,17636.072,478001888.0,141280,8794.62,238385216.0
2023-06-01 02:00:00,27065.5,27106.5,26621.3,26705.1,42682.676,1144205950.0,268982,17221.14,461549856.0
2023-06-01 03:00:00,26705.0,26829.0,26600.0,26768.6,27103.262,724444350.0,219415,14888.465,398015552.0
2023-06-01 04:00:00,26768.6,26813.6,26742.0,26797.2,9880.983,264626432.0,94780,5605.699,150137440.0
...,...,...,...,...,...,...,...,...,...
2023-07-31 20:00:00,29146.4,29249.0,29102.3,29194.5,13962.28,407463040.0,127312,7628.219,222628592.0
2023-07-31 21:00:00,29194.5,29285.3,29144.8,29230.6,10977.811,320717216.0,86779,5551.999,162209840.0
2023-07-31 22:00:00,29230.6,29232.1,29186.0,29199.6,4243.307,123939200.0,43890,1955.245,57107704.0


In [66]:
type(raw2.transform(TypedRecords())[0])

qubx.core.series.Bar

Transform to simulated ticks (Quotes)

In [68]:
raw2.transform(TickSeries())[:10]

[[2023-06-01T00:00:01.000000000]	27201.10000 (1000000000.00) | 27201.10000 (1000000000.00),
 [2023-06-01T00:24:00.000000000]	27347.90000 (1000000000.00) | 27347.90000 (1000000000.00),
 [2023-06-01T00:36:00.000000000]	27008.10000 (1000000000.00) | 27008.10000 (1000000000.00),
 [2023-06-01T00:59:59.000000000]	27059.80000 (1000000000.00) | 27059.80000 (1000000000.00),
 [2023-06-01T01:00:01.000000000]	27059.80000 (1000000000.00) | 27059.80000 (1000000000.00),
 [2023-06-01T01:24:00.000000000]	27045.70000 (1000000000.00) | 27045.70000 (1000000000.00),
 [2023-06-01T01:36:00.000000000]	27155.70000 (1000000000.00) | 27155.70000 (1000000000.00),
 [2023-06-01T01:59:59.000000000]	27065.50000 (1000000000.00) | 27065.50000 (1000000000.00),
 [2023-06-01T02:00:01.000000000]	27065.50000 (1000000000.00) | 27065.50000 (1000000000.00),
 [2023-06-01T02:24:00.000000000]	27106.50000 (1000000000.00) | 27106.50000 (1000000000.00)]

Transform to simulated ticks (Trades)

In [71]:
raw2.transform(TickSeries(trades=True, quotes=False))[:10]

[[2023-06-01T00:00:01.000000000]	27201.10000 (14532.93) ??? ,
 [2023-06-01T00:24:00.000000000]	27347.90000 (13988.44) ??? ,
 [2023-06-01T00:36:00.000000000]	27008.10000 (5118.20) ??? ,
 [2023-06-01T01:00:01.000000000]	27059.80000 (2260.62) ??? ,
 [2023-06-01T01:24:00.000000000]	27045.70000 (913.87) ??? ,
 [2023-06-01T01:36:00.000000000]	27155.70000 (14461.58) ??? ,
 [2023-06-01T02:00:01.000000000]	27065.50000 (3606.74) ??? ,
 [2023-06-01T02:24:00.000000000]	27106.50000 (31704.11) ??? ,
 [2023-06-01T02:36:00.000000000]	26621.30000 (7371.82) ??? ,
 [2023-06-01T03:00:01.000000000]	26705.00000 (12427.26) ??? ]

Transform to simulated ticks (Quotes + Trades)

In [72]:
raw2.transform(TickSeries(trades=True))[:10]

[[2023-06-01T00:00:01.000000000]	27201.10000 (1000000000.00) | 27201.10000 (1000000000.00),
 [2023-06-01T00:00:01.000000000]	27201.10000 (14532.93) ??? ,
 [2023-06-01T00:24:00.000000000]	27347.90000 (1000000000.00) | 27347.90000 (1000000000.00),
 [2023-06-01T00:24:00.000000000]	27347.90000 (13988.44) ??? ,
 [2023-06-01T00:36:00.000000000]	27008.10000 (1000000000.00) | 27008.10000 (1000000000.00),
 [2023-06-01T00:36:00.000000000]	27008.10000 (5118.20) ??? ,
 [2023-06-01T00:59:59.000000000]	27059.80000 (1000000000.00) | 27059.80000 (1000000000.00),
 [2023-06-01T01:00:01.000000000]	27059.80000 (1000000000.00) | 27059.80000 (1000000000.00),
 [2023-06-01T01:00:01.000000000]	27059.80000 (2260.62) ??? ,
 [2023-06-01T01:24:00.000000000]	27045.70000 (1000000000.00) | 27045.70000 (1000000000.00)]

Transform to OHLCV series

In [94]:
raw2.transform(OHLCVSeries(max_length=5))

                        open     high      low    close     volume  \
timestamp                                                            
2023-07-31 21:00:00  29194.5  29285.3  29144.8  29230.6  10977.811   
2023-07-31 22:00:00  29230.6  29232.1  29186.0  29199.6   4243.307   
2023-07-31 23:00:00  29199.5  29229.8  29173.4  29220.8   3765.949   
2023-08-01 00:00:00  29220.8  29277.0  29200.9  29272.3   4934.267   

                     bought_volume  volume_quote  bought_volume_quote  \
timestamp                                                               
2023-07-31 21:00:00       5551.999   320717216.0          162209840.0   
2023-07-31 22:00:00       1955.245   123939200.0           57107704.0   
2023-07-31 23:00:00       1826.605   109981072.0           53346232.0   
2023-08-01 00:00:00       2739.515   144272672.0           80101792.0   

                     trade_count  
timestamp                         
2023-07-31 21:00:00      86779.0  
2023-07-31 22:00:00      43890.0  


Transform multi symbols response

In [95]:
multi = bnc_swap.read(["BTCUSDT", "ETHUSDT"], "ohlc(1h)", None, None)

Symbol at top level column

In [124]:
multi.transform(PandasFrame())

BTCUSDT                                        \
                        open     high      low    close     volume   
timestamp                                                            
2023-06-01 00:00:00  27201.1  27347.9  27008.1  27059.8  33639.568   
2023-06-01 01:00:00  27059.8  27155.7  27045.7  27065.5  17636.072   
2023-06-01 02:00:00  27065.5  27106.5  26621.3  26705.1  42682.676   
2023-06-01 03:00:00  26705.0  26829.0  26600.0  26768.6  27103.262   
2023-06-01 04:00:00  26768.6  26813.6  26742.0  26797.2   9880.983   
...                      ...      ...      ...      ...        ...   
2023-07-31 20:00:00  29146.4  29249.0  29102.3  29194.5   13962.28   
2023-07-31 21:00:00  29194.5  29285.3  29144.8  29230.6  10977.811   
2023-07-31 22:00:00  29230.6  29232.1  29186.0  29199.6   4243.307   
2023-07-31 23:00:00  29199.5  29229.8  29173.4  29220.8   3765.949   
2023-08-01 00:00:00  29220.8  29277.0  29200.9  29272.3   4934.267   

                                                            \
                     quote_volume   count taker_buy_volume   
timestamp                                                    
2023-06-01 00:00:00   914441790.0  256082        15501.039   
2023-06-01 01:00:00   478001888.0  141280          8794.62   
2023-06-01 02:00:00  1144205950.0  268982         17221.14   
2023-06-01 03:00:00   724444350.0  219415        14888.465   
2023-06-01 04:00:00   264626432.0   94780         5605.699   
...                           ...     ...              ...   
2023-07-31 20:00:00   407463040.0  127312         7628.219   
2023-07-31 21:00:00   320717216.0   86779         5551.999   
2023-07-31 22:00:00   123939200.0   43890         1955.245   
2023-07-31 23:00:00   109981072.0   42779         1826.605   
2023-08-01 00:00:00   144272672.0   49285         2739.515   

                                            ETHUSDT                    \
                    taker_buy_quote_volume     open     high      low   
timestamp                                                               
2023-06-01 00:00:00            421503008.0  1872.85  1889.61   1867.2   
2023-06-01 01:00:00            238385216.0  1870.65  1879.05  1870.39   
2023-06-01 02:00:00            461549856.0  1873.32  1874.29   1845.0   
2023-06-01 03:00:00            398015552.0  1851.52  1857.77  1837.94   
2023-06-01 04:00:00            150137440.0  1854.81  1856.06  1851.37   
...                                    ...      ...      ...      ...   
2023-07-31 20:00:00            222628592.0  1855.84  1859.82   1851.0   
2023-07-31 21:00:00            162209840.0  1852.81  1858.12  1850.08   
2023-07-31 22:00:00             57107704.0  1856.38   1857.0  1854.17   
2023-07-31 23:00:00             53346232.0  1855.49   1857.3  1854.16   
2023-08-01 00:00:00             80101792.0   1855.4  1861.74  1854.27   

                                                               \
                       close      volume quote_volume   count   
timestamp                                                       
2023-06-01 00:00:00  1870.65  286414.624  538209090.0  208626   
2023-06-01 01:00:00  1873.32  112125.784  210286992.0   99710   
2023-06-01 02:00:00  1851.52  252476.864  468882464.0  167594   
2023-06-01 03:00:00  1854.81   218103.28  403360896.0  157489   
2023-06-01 04:00:00  1854.11   67006.124  124226136.0   66034   
...                      ...         ...          ...     ...   
2023-07-31 20:00:00  1852.82   94410.584  175123744.0   82877   
2023-07-31 21:00:00  1856.39   54486.696  100992376.0   54253   
2023-07-31 22:00:00   1855.5   25662.458   47610984.0   32540   
2023-07-31 23:00:00  1855.39   23961.846   44462552.0   28068   
2023-08-01 00:00:00  1860.99   57546.532  106932632.0   41037   

                                                             
                    taker_buy_volume taker_buy_quote_volume  
timestamp                                                    
2023-06-01 00:00:00       147333.712            276

Symbol at index 

In [125]:
multi.transform(PandasFrame(True))

open     high      low    close      volume  \
timestamp           symbol                                                    
2023-06-01 00:00:00 BTCUSDT  27201.1  27347.9  27008.1  27059.8   33639.568   
                    ETHUSDT  1872.85  1889.61   1867.2  1870.65  286414.624   
2023-06-01 01:00:00 BTCUSDT  27059.8  27155.7  27045.7  27065.5   17636.072   
                    ETHUSDT  1870.65  1879.05  1870.39  1873.32  112125.784   
2023-06-01 02:00:00 BTCUSDT  27065.5  27106.5  26621.3  26705.1   42682.676   
...                              ...      ...      ...      ...         ...   
2023-07-31 22:00:00 ETHUSDT  1856.38   1857.0  1854.17   1855.5   25662.458   
2023-07-31 23:00:00 BTCUSDT  29199.5  29229.8  29173.4  29220.8    3765.949   
                    ETHUSDT  1855.49   1857.3  1854.16  1855.39   23961.846   
2023-08-01 00:00:00 BTCUSDT  29220.8  29277.0  29200.9  29272.3    4934.267   
                    ETHUSDT   1855.4  1861.74  1854.27  1860.99   57546.532   

                             quote_volume   count taker_buy_volume  \
timestamp           symbol                                           
2023-06-01 00:00:00 BTCUSDT   914441790.0  256082        15501.039   
                    ETHUSDT   538209090.0  208626       147333.712   
2023-06-01 01:00:00 BTCUSDT   478001888.0  141280          8794.62   
                    ETHUSDT   210286992.0   99710          56460.7   
2023-06-01 02:00:00 BTCUSDT  1144205950.0  268982         17221.14   
...                                   ...     ...              ...   
2023-07-31 22:00:00 ETHUSDT    47610984.0   32540        12914.089   
2023-07-31 23:00:00 BTCUSDT   109981072.0   42779         1826.605   
                    ETHUSDT    44462552.0   28068        10439.992   
2023-08-01 00:00:00 BTCUSDT   144272672.0   49285         2739.515   
                    ETHUSDT   106932632.0   41037        34820.892   

                            taker_buy_quote_volume  
timestamp           symbol                          
2023-06-01 00:00:00 BTCUSDT            421503008.0  
                    ETHUSDT            276909632.0  
2023-06-01 01:00:00 BTCUSDT            238385216.0  
                    ETHUSDT            105882736.0  
2023-06-01 02:00:00 BTCUSDT            461549856.0  
...                                            ...  
2023-07-31 22:00:00 ETHUSDT             23957768.0  
2023-07-31 23:00:00 BTCUSDT             53346232.0  
                    ETHUSDT             19372540.0  
2023-08-01 00:00:00 BTCUSDT             80101792.0  
                    ETHUSDT             64708432.0  

[2930 rows x 9 columns]

Transform to OHLC

In [126]:
multi.transform(OHLCVSeries(max_length=3))

{'BTCUSDT':                         open     high      low    close    volume  \
 timestamp                                                           
 2023-07-31 23:00:00  29199.5  29229.8  29173.4  29220.8  3765.949   
 2023-08-01 00:00:00  29220.8  29277.0  29200.9  29272.3  4934.267   
 
                      bought_volume  volume_quote  bought_volume_quote  \
 timestamp                                                               
 2023-07-31 23:00:00       1826.605   109981072.0           53346232.0   
 2023-08-01 00:00:00       2739.515   144272672.0           80101792.0   
 
                      trade_count  
 timestamp                         
 2023-07-31 23:00:00      42779.0  
 2023-08-01 00:00:00      49285.0  ,
 'ETHUSDT':                         open     high      low    close     volume  \
 timestamp                                                            
 2023-07-31 23:00:00  1855.49  1857.30  1854.16  1855.39  23961.846   
 2023-08-01 00:00:00  1855.40  1861.74  

Transform multi chunks also supported 

In [128]:
for w in bnc_swap.read(["BTCUSDT", "ETHUSDT"], "ohlc(1h)", None, None, chunksize=1000): # type: ignore
    print('- ' * 10)
    print(w.transform(PandasFrame()).head(3))
    print('- ' * 10)
    print(w.transform(TickSeries()))

- - - - - - - - - - 
                     BTCUSDT                                        \
                        open     high      low    close     volume   
timestamp                                                            
2023-06-01 00:00:00  27201.1  27347.9  27008.1  27059.8  33639.568   
2023-06-01 01:00:00  27059.8  27155.7  27045.7  27065.5  17636.072   
2023-06-01 02:00:00  27065.5  27106.5  26621.3  26705.1  42682.676   

                                                            \
                     quote_volume   count taker_buy_volume   
timestamp                                                    
2023-06-01 00:00:00   914441790.0  256082        15501.039   
2023-06-01 01:00:00   478001888.0  141280          8794.62   
2023-06-01 02:00:00  1144205950.0  268982         17221.14   

                                            ETHUSDT                    \
                    taker_buy_quote_volume     open     high      low   
timestamp                              